In [90]:
import keras


In [91]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM, TimeDistributed, Flatten, Dropout
from keras.optimizers import Adam

In [92]:
data = np.genfromtxt('/root/sharedfolder/train.csv', delimiter=',')
train, test = data[:40000, ...].astype(int), data[40000:, ...].astype(int)

validate = np.genfromtxt('/root/sharedfolder/test.csv', delimiter=',').astype(int)

print(train.shape, test.shape, validate.shape)

(40000, 785) (2000, 785) (28000, 784)


In [93]:
train_x, train_y = train[...,1:], train[...,:1].reshape((40000))
test_x, test_y = test[...,1:], test[...,:1].reshape((2000))

print('Train features:', train_x.shape)
print('Train labels:  ', train_y.shape)
print('Test features: ', test_x.shape)
print('Test labels:   ', test_y.shape)

Train features: (40000, 784)
Train labels:   (40000,)
Test features:  (2000, 784)
Test labels:    (2000,)


In [94]:
train_placeholer = np.zeros((train_y.size, train_y.max()+1))
train_placeholer[np.arange(train_y.size),train_y] = 1
train_y = train_placeholer

test_placeholer = np.zeros((test_y.size, test_y.max()+1))
test_placeholer[np.arange(test_y.size),test_y] = 1
test_y = test_placeholer

print('Train labels:  ', train_y.shape)
print('Test labels:   ', test_y.shape)

Train labels:   (40000, 10)
Test labels:    (2000, 10)


In [96]:
train_x = train_x.reshape((40000, 28, 28, 1))
test_x = test_x.reshape((2000, 28, 28, 1))
validate = validate.reshape((28000, 28, 28, 1))

print('Train features:', train_x.shape)
print('Test features: ', test_x.shape)
print('Validate features: ', test_x.shape)

Train features: (40000, 28, 28, 1)
Test features:  (2000, 28, 28, 1)
Validate features:  (2000, 28, 28, 1)


Data done, now Model...

In [97]:
model = Sequential()
# x = Input(shape=(row, col, pixel))

model.add(Conv2D(14, (3, 3), input_shape=(28, 28, 1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(14, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


# Encodes a row of pixels using TimeDistributed Wrapper.
# encoded_rows = TimeDistributed(LSTM(row_hidden))(x)
model.add(TimeDistributed(LSTM(128, input_shape=(28,28,1))))
model.add(LSTM(128))
model.add(Dense(10))
model.add(Activation('softmax'))

In [98]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
model.fit(train_x, train_y,
          batch_size=128,
          epochs=50,
          verbose=1,
          validation_data=(test_x, test_y))

# score = model.evaluate(x_test, y_test, verbose=0)

Train on 40000 samples, validate on 2000 samples
Epoch 1/50
40000/40000 [==============================] - 293s 7ms/step - loss: 0.6690 - acc: 0.7673 - val_loss: 0.1931 - val_acc: 0.9440
Epoch 2/50
40000/40000 [==============================] - 293s 7ms/step - loss: 0.1521 - acc: 0.9537 - val_loss: 0.1575 - val_acc: 0.9525
Epoch 3/50
40000/40000 [==============================] - 297s 7ms/step - loss: 0.1031 - acc: 0.9680 - val_loss: 0.1036 - val_acc: 0.9675
Epoch 4/50
40000/40000 [==============================] - 304s 8ms/step - loss: 0.0802 - acc: 0.9759 - val_loss: 0.0843 - val_acc: 0.9710
Epoch 5/50
40000/40000 [==============================] - 299s 7ms/step - loss: 0.0659 - acc: 0.9794 - val_loss: 0.0934 - val_acc: 0.9715
Epoch 6/50
40000/40000 [==============================] - 301s 8ms/step - loss: 0.0575 - acc: 0.9826 - val_loss: 0.0664 - val_acc: 0.9805
Epoch 7/50
12288/40000 [========>.....................] - ETA: 3:21 - loss: 0.0511 - acc: 0.9839

In [87]:
predictions_raw = model.predict(validate)
predictions = np.argmax(predictions_raw, axis=1)
indexes = np.arange(predictions.shape[0])
# print(indexes[:3])

stacked = np.stack((indexes, predictions), axis=1)

print(stacked[:3])

[[0 2]
 [1 0]
 [2 9]]


In [88]:
np.savetxt("toValidate.csv", stacked.astype(int), delimiter=",")

In [89]:
csv = ''
for i,v in enumerate(stacked):
    csv+=','.join([str(i),str(int(v[1]))]) + '\n'

with open('toValidate.csv', 'w') as f:
    f.write(csv)
